# Explainability
To determine which proteins are more important for the model, we use feature importance techniques. 

In this case we are using a neural network, direct feature importance isn't as straightforward as with decision trees or linear models. However, we can approximate feature importance using techniques like:

1. Permutation Feature Importance: This method involves measuring the change in the model's performance when a single feature value is randomly shuffled.

2. SHAP (SHapley Additive exPlanations) Values: This method explains the output of a machine learning model by computing the contribution of each feature to the prediction

# Feature Names
Extract the correct feature names from the dataset.


Load and Process the Data: load the data, extract features and labels, and prepare the training and testing sets


In [3]:
import pandas as pd

# Load the Excel file
file_path = 'C:/Users/a121160/OneDrive - Eviden/Documents/Proyecto/ELMUMY/WPs/WP4/DATA/Proteomic_data_testing.xlsx'
data = pd.read_excel(file_path)

# Extract the feature names
metadata_cols = ['Accession_ID', 'Gene_Names', 'Gene_Symbol', 'Protein_Description']
feature_cols = data.columns[len(metadata_cols):]

# Extract features and labels
features = data[feature_cols].values
labels = [0] * 16 + [1] * 12 + [2] * 16

# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features.T, labels, test_size=0.2, random_state=4)

# Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to torch tensors
import torch
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)


# Define the Model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SophisticatedNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SophisticatedNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.4)
        self.fc4 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

# Model parameters
input_dim = X_train.shape[1]
output_dim = 3  # Number of classes: MGUS, sMM, MM

# Initialize the model
model = SophisticatedNN(input_dim, output_dim)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Convert data to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy:.4f}')


C:\Users\a121160\AppData\Local\Temp\ipykernel_31276\3374194350.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
C:\Users\a121160\AppData\Local\Temp\ipykernel_31276\3374194350.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float32)
C:\Users\a121160\AppData\Local\Temp\ipykernel_31276\3374194350.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
C:\Users\a121160\AppData\Local\Temp\ipyker

Epoch [10/100], Loss: 0.7759
Epoch [20/100], Loss: 0.5174
Epoch [30/100], Loss: 0.3706
Epoch [40/100], Loss: 0.2343
Epoch [50/100], Loss: 0.1374
Epoch [60/100], Loss: 0.1096
Epoch [70/100], Loss: 0.0910
Epoch [80/100], Loss: 0.0825
Epoch [90/100], Loss: 0.0374
Epoch [100/100], Loss: 0.0270
Accuracy: 0.6667


# Train the Model


In [5]:
# Ensure the model is in evaluation mode


# Model parameters
input_dim = X_train.shape[1]
output_dim = 3  # Number of classes: MGUS, sMM, MM

# Initialize the model
model = SophisticatedNN(input_dim, output_dim)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop with loss recording
num_epochs = 100
loss_values = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    loss_values.append(loss.item())

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.7057
Epoch [20/100], Loss: 0.4069
Epoch [30/100], Loss: 0.3493
Epoch [40/100], Loss: 0.1917
Epoch [50/100], Loss: 0.1873
Epoch [60/100], Loss: 0.1221
Epoch [70/100], Loss: 0.0841
Epoch [80/100], Loss: 0.0573
Epoch [90/100], Loss: 0.0501
Epoch [100/100], Loss: 0.0237


# Compute SHAP Values and Plot
Intall "shap" librery

In [6]:
! pip install shap matplotlib

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\a121160\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Compute SHAP values and plot:

Using SHAP's DeepExplainer, we compute the SHAP values for each class
We plot the SHAP summary plots for each class (MGUS, sMM, MM) with the correct feature names.

In [7]:
import shap
import torch
import numpy as np
import matplotlib.pyplot as plt

# Convert data to numpy for SHAP
X_train_np = X_train.detach().cpu().numpy()
X_test_np = X_test.detach().cpu().numpy()

# Ensure the model is in evaluation mode
model.eval()

# Define a function to wrap the model prediction to be compatible with SHAP
def model_predict(data):
    tensor_data = torch.tensor(data, dtype=torch.float32)
    with torch.no_grad():
        return model(tensor_data).numpy()

# Use SHAP KernelExplainer instead of DeepExplainer for PyTorch models
explainer = shap.KernelExplainer(model_predict, X_train_np[:100])  # Using a subset for the background
shap_values = explainer.shap_values(X_test_np, nsamples=100)  # Adjust nsamples for performance


# Plot summary plot for the first class (MGUS)
shap.summary_plot(shap_values[0], X_test_np, feature_names=feature_cols, show=False)
plt.title('SHAP Summary Plot for Class 0 (MGUS)')
plt.show()

# Plot summary plot for the second class (sMM)
shap.summary_plot(shap_values[1], X_test_np, feature_names=feature_cols, show=False)
plt.title('SHAP Summary Plot for Class 1 (sMM)')
plt.show()

# Plot summary plot for the third class (MM)
shap.summary_plot(shap_values[2], X_test_np, feature_names=feature_cols, show=False)
plt.title('SHAP Summary Plot for Class 2 (MM)')
plt.show()



 44%|████▍     | 4/9 [00:01<00:01,  3.47it/s]C:\Users\a121160\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_least_angle.py:718: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 40 iterations, alpha=3.277e-03, previous alpha=3.268e-03, with an active set of 31 regressors.
  warnings.warn(
C:\Users\a121160\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_least_angle.py:688: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 22 iterations, i.e. alpha=1.321e-02, with an active set of 20 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
C:\Users\a121160\AppData\Local\Packages\PythonSoftwareFoundatio

AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

# Permuation Feature Importance

This method evaluates the importance of each feature by measuring the increase in the model's prediction error when the feature's values are randomly shuffled. 
The larger the increase in error, the more important the feature is.

Permutation Feature Importance
We'll use sklearn's permutation_importance function to measure the importance of each feature.

Permutation Feature Importance: This method evaluates the importance of each feature by measuring the increase in the model's prediction error when the feature's values are randomly shuffled.
 The larger the increase in error, the more important the feature is

In [6]:
from sklearn.inspection import permutation_importance

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy:.4f}')

# Use permutation importance
def model_predict(X):
    with torch.no_grad():
        X_tensor = torch.tensor(X, dtype=torch.float32)
        outputs = model(X_tensor)
        _, predicted = torch.max(outputs, 1)
        return predicted.numpy()

results = permutation_importance(model_predict, X_test.numpy(), y_test.numpy(), n_repeats=10, random_state=44)
importance = results.importances_mean

# Plot feature importances
plt.figure(figsize=(12, 8))
plt.bar(range(len(importance)), importance)
plt.title('Permutation Feature Importance')
plt.xlabel('Feature Index')
plt.ylabel('Importance')
plt.show()


Accuracy: 1.0000


InvalidParameterError: The 'estimator' parameter of permutation_importance must be an object implementing 'fit'. Got <function model_predict at 0x0000023334A8E160> instead.